In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
import time
import random
import pandas as pd
from tkinter import simpledialog, Tk
import re
from datetime import datetime
from urllib.parse import urlparse, parse_qs, urlunparse, urlencode

# 웹드라이버 옵션 설정
options = Options()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36")

# 페이지 번호를 업데이트하는 함수
def update_url_page(url, page_number):
    url_parts = list(urlparse(url))
    query = parse_qs(url_parts[4])
    query['page'] = [page_number]
    url_parts[4] = urlencode(query, doseq=True)
    return urlunparse(url_parts)

# 데이터 변환 및 검증 함수
def convert_price(price_str):
    if not price_str or price_str in ["No base price available", "No price available"]:
        return None
    cleaned_price = price_str.replace(",", "").replace("원", "").strip()
    try:
        return float(cleaned_price)
    except ValueError:
        return None

def convert_review_count(review_count_str):
    if not review_count_str or review_count_str == "(리뷰 없음)":
        return 0
    cleaned_count = review_count_str.replace("(", "").replace(")", "").strip()
    try:
        return int(cleaned_count)
    except ValueError:
        return 0

def is_advertisement(product):
    try:
        product.find_element(By.CLASS_NAME, "ad-badge-text")
        return "예"
    except NoSuchElementException:
        return "아니오"

def extract_vendor_item_id(url):
    match = re.search(r"vendorItemId=(\d+)", url)
    return match.group(1) if match else "No vendorItemId"

def determine_delivery_type(product):
    try:
        badge = product.find_element(By.CLASS_NAME, "badge.rocket")
        img_src = badge.find_element(By.TAG_NAME, "img").get_attribute("src")
        if "logoRocketMerchantLargeV3R3" in img_src:
            return "판매자로켓"
        elif "rocketwow-bi-16" in img_src:
            return "로켓와우"
        elif "logo_rocket_large" in img_src:
            return "로켓배송"
    except NoSuchElementException:
        pass
    
    try:
        global_badge = product.find_element(By.CLASS_NAME, "badge.global")
        img_src = global_badge.find_element(By.TAG_NAME, "img").get_attribute("src")
        if "global_b" in img_src:
            return "로켓직구"
    except NoSuchElementException:
        pass
    
    return "일반배송"

def collect_product_info():
    products = driver.find_elements(By.CSS_SELECTOR, '.search-product')
    images = driver.find_elements(By.CLASS_NAME, "search-product-wrap-img")
    image_urls = [image.get_attribute('src') for image in images if image.get_attribute('src') is not None]

    for index, product in enumerate(products, start=1):
        try:
            name = product.find_element(By.CLASS_NAME, "name").text
        except NoSuchElementException:
            name = "No name available"
        
        advertisement = is_advertisement(product)
        
        try:
            base_price = convert_price(product.find_element(By.CLASS_NAME, "base-price").text)
        except NoSuchElementException:
            base_price = None
        
        try:
            price = convert_price(product.find_element(By.CLASS_NAME, "price-value").text)
        except NoSuchElementException:
            price = None
        
        try:
            review_count = convert_review_count(product.find_element(By.CLASS_NAME, "rating-total-count").text)
        except NoSuchElementException:
            review_count = 0
        
        try:
            product_url = product.find_element(By.CSS_SELECTOR, "a").get_attribute('href')
            vendor_item_id = extract_vendor_item_id(product_url)
        except NoSuchElementException:
            product_url = "No URL available"
            vendor_item_id = "No vendorItemId"

        image_url = image_urls[index-1] if index-1 < len(image_urls) else "No image available"

        delivery_type = determine_delivery_type(product)

        data.append([vendor_item_id, index, name, advertisement, base_price, price, review_count, product_url, image_url, delivery_type])
        print(f"Collected data for product {index}: {name}")  # 데이터 수집 과정 출력
        
        
# GUI 입력 및 초기화
root = Tk()
root.withdraw()
url = simpledialog.askstring("URL 입력", "쿠팡 상품 검색 결과 페이지 URL을 입력하세요:")
if not url:
    print("URL이 입력되지 않았습니다.")
    exit()

today_str = datetime.now().strftime("%Y%m%d")
excel_filename = simpledialog.askstring("파일명 입력", "저장할 엑셀 파일명을 입력하세요 (확장자 제외):")
if not excel_filename:
    excel_filename = "쿠팡상품검색크롤링"
excel_filename += f"_{today_str}.xlsx"

# 웹드라이버 초기화 및 URL 접속
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
data = []

# 데이터 수집
for page in range(1, 2):
    updated_url = update_url_page(url, page) if page > 1 else url
    driver.get(updated_url)
    time.sleep(random.uniform(3, 6))  # 페이지 로딩 대기
    print(f"Processing page {page}")
    collect_product_info()

# 데이터프레임 생성 및 엑셀 파일로 저장
df = pd.DataFrame(data, columns=['VendorItemId', '순번', '상품명', '광고여부', '기본가격', '판매가', '리뷰수', '상품URL', '이미지URL', '배송형태'])
df.to_excel(excel_filename, index=False)
print(f"{excel_filename}로 저장되었습니다.")

# 웹드라이버 종료
driver.quit()

Processing page 1
1_20251001.xlsx로 저장되었습니다.
